# Proyecto 02 del 1er Bimestre
## Analítica Presciptiva

### Tarea 1: Importación de datos, limpieza y preprocesamiento

Se debe centrar en importar los datos proporcionados en un formato adecuado para que el análisis posterior sea
más sencillo. También verificar si es necesario realizar alguna limpieza de datos y/o pasos de preprocesamiento.

Importación de librerias

In [1]:
import numpy as np
import pandas as pd

Se crea la función para calcular el AQI en base a The EPA's table of breakpoints.  

|O3(ppb)|O3(ppb)|PM2.5(ug/m3)|PM10(ug/m3)|CO(ppm)|SO2(ppb)|NO2(ppb)|AQI|
|-------|-------|------------|-----------|-------|--------|--------|---|
|0–54|-|0.0–12.0|0–54|0.0–4.4|0–35|0–53|0–50|
|(8-hr)||(24-hr)|(24-hr)|(8-hr)|(1-hr)|(1-hr)|
|55–70|-|12.1–35.4|55–154|4.5–9.4|36–75|54–100|51–100|
|(8-hr)||(24-hr)|(24-hr)|(8-hr)|(1-hr)|(1-hr)|
|71–85|125–164|35.5–55.4|155–254|9.5–12.4|76–185|101–360|101–150|
|(8-hr)|(1-hr)|(24-hr)|(24-hr)|(8-hr)|(1-hr)|(1-hr)||
|86–105|165–204|55.5–150.4|255–354|12.5–15.4|186–304|361–649|151–200|
|(8-hr)|(1-hr)|(24-hr)|(24-hr)|(8-hr)|(1-hr)|(1-hr)||
|106–200|205–404|150.5–250.4|355–424|15.5–30.4|305–604|650–1249|201–300|
|(8-hr)|(1-hr)|(24-hr)|(24-hr)|(8-hr)|(24-hr)|(1-hr)||
|—|405–504|250.5–350.4|425–504|30.5–40.4|605–804|1250–1649|301–400|
||(1-hr)|(24-hr)|(24-hr)|(8-hr)|(24-hr)|(1-hr)||
|—|505–604|350.5–500.4|505–604|40.5–50.4|805–1004|1650–2049|401–500|
||(1-hr)|(24-hr)|(24-hr)|(8-hr)|(24-hr)|(1-hr)||
  
Fuente: https://en.wikipedia.org/wiki/Air_quality_index

  
Estos valores fueron transformados de ppb o ppm a ug/m3 en base a la siguiente tabla de conversión:

|Air Pollutant|Conversion Factor|
|-------------|-----------------|
|Sulphur dioxide (SO2)  |1 ppb = 2.62 µg/m3  |
|Nitrogen dioxide (NO2) |1 ppb = 1.88 µg/m3  |
|Carbon monoxide (CO)   |1 ppb = 1.15 µg/m3  |
|Ozone (O3)             |1 ppb = 1.96 µg/m3  |
  
Fuente: https://www.breeze-technologies.de/blog/air-pollution-how-to-convert-between-mgm3-%C2%B5gm3-ppm-ppb/

In [2]:
def calculate_aqi(concentration, pollutant):
   if pollutant == "PM10":
      breakpoints = [0, 54, 154, 254, 354, 424, 504, 604]
   elif pollutant == "SO2":
      breakpoints = [0, 91.7, 196.5, 484.7, 796.48, 1582.48, 2630.48]
   elif pollutant == "NO2":
      breakpoints = [0, 99.64, 188, 676.8, 1220.12, 2348.12, 3852.12]
   elif pollutant == "CO":
      breakpoints = [0, 5060, 10810, 14260, 17710, 34960, 57960]
   elif pollutant == "O3":
      breakpoints = [0, 105.84, 137.2, 166.6, 205.8, 392, 604]
   else:
      return "Invalid pollutant"
   
   index = [0, 50, 100, 150, 200, 300, 400, 500]

   for i in range(len(breakpoints) - 1):
      if concentration >= breakpoints[i] and concentration <= breakpoints[i + 1]:
         aqi = ((index[i + 1] - index[i]) / (breakpoints[i + 1] - breakpoints[i])) * (concentration - breakpoints[i]) + index[i]
         return round(aqi)

   return 500

Se lee el archivo csv a dataframe

In [3]:
df = pd.read_csv(filepath_or_buffer='../data/PRSA_Data_20130301-20170228/PRSA_Data_Aotizhongxin_20130301-20170228.csv',sep=',')
df2 = pd.read_csv(filepath_or_buffer='../data/PRSA_Data_20130301-20170228/PRSA_Data_Changping_20130301-20170228.csv',sep=',')
df3 = pd.read_csv(filepath_or_buffer='../data/PRSA_Data_20130301-20170228/PRSA_Data_Dingling_20130301-20170228.csv',sep=',')
df4 = pd.read_csv(filepath_or_buffer='../data/PRSA_Data_20130301-20170228/PRSA_Data_Dongsi_20130301-20170228.csv',sep=',')

df = pd.concat([df,df2,df3,df4])

df.head(25)

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
0,1,2013,3,1,0,4.0,4.0,4.0,7.0,300.0,77.0,-0.7,1023.0,-18.8,0.0,NNW,4.4,Aotizhongxin
1,2,2013,3,1,1,8.0,8.0,4.0,7.0,300.0,77.0,-1.1,1023.2,-18.2,0.0,N,4.7,Aotizhongxin
2,3,2013,3,1,2,7.0,7.0,5.0,10.0,300.0,73.0,-1.1,1023.5,-18.2,0.0,NNW,5.6,Aotizhongxin
3,4,2013,3,1,3,6.0,6.0,11.0,11.0,300.0,72.0,-1.4,1024.5,-19.4,0.0,NW,3.1,Aotizhongxin
4,5,2013,3,1,4,3.0,3.0,12.0,12.0,300.0,72.0,-2.0,1025.2,-19.5,0.0,N,2.0,Aotizhongxin
5,6,2013,3,1,5,5.0,5.0,18.0,18.0,400.0,66.0,-2.2,1025.6,-19.6,0.0,N,3.7,Aotizhongxin
6,7,2013,3,1,6,3.0,3.0,18.0,32.0,500.0,50.0,-2.6,1026.5,-19.1,0.0,NNE,2.5,Aotizhongxin
7,8,2013,3,1,7,3.0,6.0,19.0,41.0,500.0,43.0,-1.6,1027.4,-19.1,0.0,NNW,3.8,Aotizhongxin
8,9,2013,3,1,8,3.0,6.0,16.0,43.0,500.0,45.0,0.1,1028.3,-19.2,0.0,NNW,4.1,Aotizhongxin
9,10,2013,3,1,9,3.0,8.0,12.0,28.0,400.0,59.0,1.2,1028.5,-19.3,0.0,N,2.6,Aotizhongxin


Se limpian las columnas que no son necesarias.  
Se concatena year-month-day con el siguiente formato AAAA-MM-DD para manejar de mejor manera los datos para agrupar por día.

In [4]:
df = df.drop(columns=['No','hour','PM2.5','TEMP','PRES','DEWP','RAIN','wd','WSPM'], errors='ignore')

combine_lambda = lambda x: '{}-{:02d}-{:02d}'.format(x.year, x.month, x.day)

# df['date'] = df['year'].astype(str) + '-' + df['month'].astype(str)

df['date'] = df.apply(combine_lambda, axis = 1)

# df = df.drop(columns=['year','month','day'], errors='ignore')

df.head(25)

,year,month,day,PM10,SO2,NO2,CO,O3,station,date
0,2013,3,1,4.0,4.0,7.0,300.0,77.0,Aotizhongxin,2013-03-01
1,2013,3,1,8.0,4.0,7.0,300.0,77.0,Aotizhongxin,2013-03-01
2,2013,3,1,7.0,5.0,10.0,300.0,73.0,Aotizhongxin,2013-03-01
3,2013,3,1,6.0,11.0,11.0,300.0,72.0,Aotizhongxin,2013-03-01
4,2013,3,1,3.0,12.0,12.0,300.0,72.0,Aotizhongxin,2013-03-01
5,2013,3,1,5.0,18.0,18.0,400.0,66.0,Aotizhongxin,2013-03-01
6,2013,3,1,3.0,18.0,32.0,500.0,50.0,Aotizhongxin,2013-03-01
7,2013,3,1,6.0,19.0,41.0,500.0,43.0,Aotizhongxin,2013-03-01
8,2013,3,1,6.0,16.0,43.0,500.0,45.0,Aotizhongxin,2013-03-01
9,2013,3,1,8.0,12.0,28.0,400.0,59.0,Aotizhongxin,2013-03-01


Se verifica si hay valores nulos o NaN.

In [5]:
df.isnull().sum()

year          0
month         0
day           0
PM10       2509
SO2        2956
NO2        4525
CO         8506
O3         4201
station       0
date          0
dtype: int64

Se limpia los valores nulos o NaN.  
Se vuelve a verificar.

In [6]:
df = df.dropna()

df.isnull().sum()

year       0
month      0
day        0
PM10       0
SO2        0
NO2        0
CO         0
O3         0
station    0
date       0
dtype: int64

Con la nueva columna "date" se agrupan los datos.  
Se calcula el mean de MP10, SO2 y NO2 y el max de CO y O3.

In [7]:
df2 = df.groupby(['date','station']).agg(
    PM10=('PM10','mean'),
    SO2 = ('SO2','mean'),
    NO2 = ('NO2','mean'),
    CO = ('CO','max'),
    O3 = ('O3','max')
    ).reset_index()

df2



,date,station,PM10,SO2,NO2,CO,O3
0,2013-03-01,Aotizhongxin,10.750000,11.708333,22.583333,700.0,77.0
1,2013-03-01,Changping,18.958333,16.041667,15.333333,700.0,94.0
2,2013-03-01,Dingling,7.250000,3.000000,2.625000,300.0,84.0
3,2013-03-01,Dongsi,10.666667,8.714286,29.571429,700.0,92.0
4,2013-03-02,Aotizhongxin,42.083333,36.625000,66.666667,1700.0,52.0
...,...,...,...,...,...,...,...
5590,2017-02-27,Dongsi,131.250000,22.000000,87.150000,3400.0,74.0
5591,2017-02-28,Aotizhongxin,23.291667,9.125000,38.291667,1400.0,122.0
5592,2017-02-28,Changping,25.363636,7.818182,26.318182,1300.0,72.0
5593,2017-02-28,Dingling,21.454545,6.363636,10.818182,800.0,102.0


Con una función lambda se llama a la función para calcular el AQI de cada gas previamente preprocesado.

In [8]:

aqi_lambda_pm10 = lambda x: calculate_aqi(x.PM10, 'PM10')
aqi_lambda_so2 = lambda x: calculate_aqi(x.SO2, 'SO2')
aqi_lambda_no2 = lambda x: calculate_aqi(x.NO2, 'NO2')
aqi_lambda_co = lambda x: calculate_aqi(x.CO, 'CO')
aqi_lambda_o3 = lambda x: calculate_aqi(x.O3, 'O3')

df2['I_PM10'] = df2.apply(aqi_lambda_pm10,axis=1)
df2['I_SO2'] = df2.apply(aqi_lambda_so2,axis=1)
df2['I_NO2'] = df2.apply(aqi_lambda_no2,axis=1)
df2['I_CO'] = df2.apply(aqi_lambda_co,axis=1)
df2['I_O3'] = df2.apply(aqi_lambda_o3,axis=1)

df2.head(10)



,date,station,PM10,SO2,NO2,CO,O3,I_PM10,I_SO2,I_NO2,I_CO,I_O3
0,2013-03-01,Aotizhongxin,10.750000,11.708333,22.583333,700.0,77.0,10,6,11,7,36
1,2013-03-01,Changping,18.958333,16.041667,15.333333,700.0,94.0,18,9,8,7,44
2,2013-03-01,Dingling,7.250000,3.000000,2.625000,300.0,84.0,7,2,1,3,40
3,2013-03-01,Dongsi,10.666667,8.714286,29.571429,700.0,92.0,10,5,15,7,43
4,2013-03-02,Aotizhongxin,42.083333,36.625000,66.666667,1700.0,52.0,39,20,33,17,25
5,2013-03-02,Changping,43.095238,43.428571,61.904762,2000.0,69.0,40,24,31,20,33
6,2013-03-02,Dingling,41.708333,25.083333,56.375000,1200.0,75.0,39,14,28,12,35
7,2013-03-02,Dongsi,40.583333,28.291667,58.500000,2600.0,62.0,38,15,29,26,29
8,2013-03-03,Aotizhongxin,120.541667,61.291667,81.000000,2799.0,77.0,83,33,41,28,36
9,2013-03-03,Changping,82.666667,37.875000,58.791667,2399.0,82.0,64,21,30,24,39


Con los valores de AQI de cada Gas se calcula el valor Final de AQI.

In [9]:
df2['AQI'] = df2[['I_PM10','I_SO2','I_NO2','I_CO','I_O3']].max(axis=1)

df2

,date,station,PM10,SO2,NO2,CO,O3,I_PM10,I_SO2,I_NO2,I_CO,I_O3,AQI
0,2013-03-01,Aotizhongxin,10.750000,11.708333,22.583333,700.0,77.0,10,6,11,7,36,36
1,2013-03-01,Changping,18.958333,16.041667,15.333333,700.0,94.0,18,9,8,7,44,44
2,2013-03-01,Dingling,7.250000,3.000000,2.625000,300.0,84.0,7,2,1,3,40,40
3,2013-03-01,Dongsi,10.666667,8.714286,29.571429,700.0,92.0,10,5,15,7,43,43
4,2013-03-02,Aotizhongxin,42.083333,36.625000,66.666667,1700.0,52.0,39,20,33,17,25,39
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5590,2017-02-27,Dongsi,131.250000,22.000000,87.150000,3400.0,74.0,89,12,44,34,35,89
5591,2017-02-28,Aotizhongxin,23.291667,9.125000,38.291667,1400.0,122.0,22,5,19,14,76,76
5592,2017-02-28,Changping,25.363636,7.818182,26.318182,1300.0,72.0,23,4,13,13,34,34
5593,2017-02-28,Dingling,21.454545,6.363636,10.818182,800.0,102.0,20,3,5,8,48,48


Con el valor Final de AQI se añade el Nivel de Contaminación del Aire.

In [10]:
niveles = [0,50,100,150,200,250,300,600]
labels = ['Excelent','Good','Slightly Polluted','Lightly Polluted','Moderately Polluted','Heavily Polluted','Severely Polluted']

df2['AIR_LEVEL'] = pd.cut(df2.AQI,niveles,labels=labels)

df2.head(20)

,date,station,PM10,SO2,NO2,CO,O3,I_PM10,I_SO2,I_NO2,I_CO,I_O3,AQI,AIR_LEVEL
0,2013-03-01,Aotizhongxin,10.750000,11.708333,22.583333,700.0,77.0,10,6,11,7,36,36,Excelent
1,2013-03-01,Changping,18.958333,16.041667,15.333333,700.0,94.0,18,9,8,7,44,44,Excelent
2,2013-03-01,Dingling,7.250000,3.000000,2.625000,300.0,84.0,7,2,1,3,40,40,Excelent
3,2013-03-01,Dongsi,10.666667,8.714286,29.571429,700.0,92.0,10,5,15,7,43,43,Excelent
4,2013-03-02,Aotizhongxin,42.083333,36.625000,66.666667,1700.0,52.0,39,20,33,17,25,39,Excelent
5,2013-03-02,Changping,43.095238,43.428571,61.904762,2000.0,69.0,40,24,31,20,33,40,Excelent
6,2013-03-02,Dingling,41.708333,25.083333,56.375000,1200.0,75.0,39,14,28,12,35,39,Excelent
7,2013-03-02,Dongsi,40.583333,28.291667,58.500000,2600.0,62.0,38,15,29,26,29,38,Excelent
8,2013-03-03,Aotizhongxin,120.541667,61.291667,81.000000,2799.0,77.0,83,33,41,28,36,83,Good
9,2013-03-03,Changping,82.666667,37.875000,58.791667,2399.0,82.0,64,21,30,24,39,64,Good


### Tarea 2: Análisis exploratorio de datos

Resumir y visualizar los datos de las formas útiles. Los estudiantes deben pensar en preguntas interesantes que
se podrían comprobar con los datos disponibles y dar respuestas textuales o mediante visualización de datos.

### Tarea 3: Modelado predictivo

Se debe definir una tarea predictiva que pueda ayudar a predecir la contaminación del aire, a través del valor del
AQI o del Nivel de Contaminación del Aire, en función de sus características. Después de definir la tarea, se debe
utilizar los datos disponibles para seleccionar y obtener un buen modelo para esta tarea. Se debe justificar el
modelo sugerido.

In [11]:
df_final = df2[df2['station'] == 'Aotizhongxin']

In [29]:
from sklearn.model_selection import train_test_split

X = df_final[['I_PM10','I_SO2','I_NO2','I_CO','I_O3']]
y = df_final['AIR_LEVEL']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [30]:
from sklearn.svm import SVC

model = SVC(kernel='linear')
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

In [31]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='micro')
recall = recall_score(y_test, y_pred, average='micro')
f1 = f1_score(y_test, y_pred, average='micro')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Accuracy: 0.8041958041958042
Precision: 0.8041958041958042
Recall: 0.8041958041958042
F1 Score: 0.8041958041958042


In [161]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(model, X, y, cv=5)
print("Cross-Validation Scores:", scores)
print("Average Accuracy:", scores.mean())

Cross-Validation Scores: [0.94055944 0.87762238 0.92982456 0.9122807  0.89473684]
Average Accuracy: 0.9110047846889952


In [162]:
from sklearn.model_selection import GridSearchCV

param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
grid_search = GridSearchCV(model, param_grid, cv=4)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_model = grid_search.best_estimator_
print('Best parameters', best_params)
print('Best models', best_model)

Best parameters {'C': 10, 'kernel': 'rbf'}
Best models SVC(C=10)


In [165]:
station = 'Dongsi'
df2.query('station == @station')[['date','I_PM10','I_SO2','I_NO2','I_CO','I_O3','AQI','AIR_LEVEL']]

,date,I_PM10,I_SO2,I_NO2,I_CO,I_O3,AQI,AIR_LEVEL
3,2013-03-01,10,5,15,7,43,43,Excelent
7,2013-03-02,38,15,29,26,29,38,Excelent
11,2013-03-03,89,32,47,33,44,89,Good
15,2013-03-04,38,9,25,40,48,48,Excelent
19,2013-03-05,118,45,76,32,48,118,Slightly Polluted
...,...,...,...,...,...,...,...,...
5578,2017-02-24,44,7,24,21,41,44,Excelent
5582,2017-02-25,21,3,15,9,43,43,Excelent
5586,2017-02-26,45,5,25,9,37,45,Excelent
5590,2017-02-27,89,12,44,34,35,89,Good


In [166]:
date = '2013-03-05'

# df_test = pd.DataFrame({
#     'I_PM10': [92],
#     'I_SO2': [26],
#     'I_NO2': [27],
#     'I_CO': [17],
#     'I_O3': [21]
#     })

df_test = df2.query('date == @date and station == @station')[['I_PM10','I_SO2','I_NO2','I_CO','I_O3']].head(1)

model.predict(df_test)

array(['Slightly Polluted'], dtype=object)